<a href="https://colab.research.google.com/github/CatarinaRRF/IC_design_de_siRNA/blob/main/code/IC_design_de_siRNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><font color=2C3E50> IC - Design de siRNA</a></font></b>
<hr color ='2C3E50' size="10">

## <font color=2C3E50><b> 01. Introdução</b>

Com a crescente importância das pesquisas em biologia molecular e terapia genética, a identificação e seleção eficiente de pequenas moléculas de RNA interferente (siRNA) tornaram-se cruciais para a regulação precisa da expressão gênica.

Nesse contexto, se faz necessario a criação de algoritmo capaz de automatizar e aprimorar o processo de escolha de siRNA. Ao integrar dados genômicos, informações sobre o alvo gênico e considerações terapêuticas, o algoritmo visa simplificar a complexidade inerente à seleção de siRNA, oferecendo aos pesquisadores uma ferramenta confiável para acelerar suas investigações e aplicações práticas.

O algoritimo utiliza do banco de dados mantido pelo NCBI <i>gene</i> onde, a partir dos codigo do gene de interesse fornecido pelo usuario, será analizado a viabilidade dos possiveis siRNA produzidos. Segue o desenho esquematico da logica do programa.


<img style="display: inline; margin: 15px 0;" title="heartica_logo" src= https://raw.githubusercontent.com/CatarinaRRF/IC_design_de_siRNA/main/media/siRNA.drawio%20.png alt="" />

## <font color=2C3E50><b> 02.Buscando o Codigo</b>



Bloco de código onde o usuário entra com o nome do genes (HOXB7) a pesquisar no portal NCBI, após a pesquisa, será exibido detalhes do genes, e será salvo automaticamente no formato fasta

email = catarinafreitas1605@gmail.com

<font color=82D712><b>Importando Bibliotecas

In [100]:
pip install biopython

In [101]:
import Bio
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq

<font color=82D712><b> Buscando os dados

In [102]:
def search_and_save(gene_name, email):
    Entrez.email = email
    handle = Entrez.esearch(db="gene", term=gene_name)
    record = Entrez.read(handle)
    handle.close()

    if record["Count"] == "0":
        print("Nenhum gene encontrado com o nome fornecido.")
        return

    id_list = record["IdList"]
    handle = Entrez.efetch(db="nucleotide", id=id_list[0], rettype="fasta", retmode="text")
    seq_record = SeqIO.read(handle, "fasta")
    handle.close()

    SeqIO.write(seq_record, f"{gene_name}.fasta", "fasta")

    #print(f"Nome: {seq_record.name}")
    #print(f"ID: {seq_record.id}")
    #print(f"Descrição: {seq_record.description}")
    #print(f"Tamanho: {len(seq_record)}")
    #print(f"Sequência: {seq_record.seq}")
    return seq_record

#gene_name = input("Digite o nome do gene que deseja buscar: ")
#email = input("Digite o seu email: ")
gene_name = "HOXB7"
email = "catarinafreitas1605@gmail.com"

seq_record = search_and_save(gene_name, email)
seq_record

SeqRecord(seq=Seq('AGTACAAGGGCCAATTACAGAGGCTTCGAATGGTGGACAGGCACCGAGTGGAGC...TTA'), id='AACY021969827.1', name='AACY021969827.1', description='AACY021969827.1 Marine metagenome 1091140274210, whole genome shotgun sequence', dbxrefs=[])

# <font color=2C3E50><b> 03.Construção dos possiveis siRNA</b>
<hr>


Bloco de codigo onde se vai trascrever o codigo, e retirar as sequencias possiveis de miRNA

<font color=82D712><b>Importando Bibliotecas

# <font color=2C3E50><b> 04. Classificação dos siRNA</b>
<hr>

Os criterios determinados por Reynolds et al. (2004), Ui-Tei et al. (2004) e Amarzguioui et al. são o Conteúdo CG%, temperatura de melting e a estabilidade interna baixa.

Caso estes criterios sejam respeitados as sequências são funcionais. Os ranking de importancia para estes criterios são os seguintes:

|Rank|Critérios|score|
|----|---------|-----|
|1|Ausência de repetições invertidas que formariam hairpin (Tm)| - |
|2| Estabilidade interna baixa | 2|
|3| Baixo GC | 1|
|4| Posições específicas | 1|

Dessa forma, os siRNA candidatos iram passar por esses critérios para se determinar quais tem mais chance de se tornarem viaveis. Para o primeiro criterio os siRNA que não se enquadrarem seram descartados, para os demais será atribuido um score, para aqueles que entrarem em conformidade com o 2° criterio receberam um score de 2, conformidade com o 3 e o 4 daram 1 ponto.

É importante ressaltar que no caso das posições especificas cada posição dará 1 ponto, para aquelas que possuem posições onde há um nucleotido ideial, a não confomidade retribuirá 0 pts, já nas posições onde o nucleotido precisa ser especifico será dado -1 pt.

O siRNA ideal deve então possuir um total de pontos entre 3 < siRNA_score < 8

> <font color=Red>Tem que concertar a Estabilidade interna baixa, o criterio ta multiplicando o 2 para cada letra no intervalo. </font>

<hr>

> <font size="2.5"> codigo usado como base de biologyguy in <a href="https://github.com/biologyguy/public_scripts/blob/master/siRNA_predict.py" rel="">github</a>

<font color=82D712><b>Importando Bibliotecas

In [103]:
import re
from Bio.SeqUtils import MeltingTemp as mt
from Bio.SeqUtils import gc_fraction
from Bio.Seq import Seq

<font color=82D712><b>Determinando a qualidade do siRNA

In [104]:
def siRNA_score (sequence):
  score = 0

  # Ausência de repetições invertidas (Tm_GC)
  tm = round(mt.Tm_GC(sequence, strict=False), 2)
  #tm_antisenso = round(mt.Tm_GC(sequence.complement(), strict=False), 2)
  #score_termodynamic = tm - tm_antisenso
  if tm > 0:
    print("não é um siRNA funcional") #trocar para delete() quando a lista de rna for produzida

  # Estabilidade interna baixa
  for letra in sequence[14:19]:
    if letra == "A" or letra == "U":
      score += 2
    else:
      score -= 2

  # Conteudo Baixo GC
  #conteudo_gc = round((len(re.findall("C", sequence[14:]))/len(re.findall("G", sequence[14:])))*100, 2)
  conteudo_gc = gc_fraction(sequence)*100
  if 30 <= conteudo_gc <= 52:
    score += 1

  # Posições específicas - Critérios REYNOLDS et al. (2004) e AMARZGUIOUI et al. (2004)
  if seq_record.seq[0] == "A" or seq_record.seq[9] != "U":
    score += 1
  if seq_record.seq[5] == "A":
    score += 1
  else: score -= 1
  if seq_record.seq[9] == "U":
    score += 1
  if seq_record.seq[9] != "G":
    score += 1
  else: score -= 1
  if seq_record.seq[18] != "G" and seq_record.seq[18] != "C":
    score += 1
  else: score -= 1
  if seq_record.seq[18] == "A":
    score += 1
  else: score -= 1

  return conteudo_gc, tm, score

siRNA_score (seq_record)

não é um siRNA funcional


(45.750262329485835, 73.98, 2)

# <font color=2C3E50><b>05. Blast</b>



Essa parte é especializada em trazer as informações BLAST do arquivo fasta, porém o processamento é online, demorando alguns minutos <b>[valdeir]</b>

<font color=82D712><b>Importando Bibliotecas